## Publish your Collection using the EDC / BC Geoservice

Publish you collections as WMS service. The BC geoservice provides access to publishing your collection as a WMS service using the geoDB Python client. Please refer to the [geoserver](https://docs.geoserver.org/latest/en/user/services/wms/reference.html) documentation for configuring the rest API call to your WMS/geojson service. 

You will have access to this service if you have purchased a large management account. This service is in beta mode. If you find any issues, please report them on our [GitHub Issues page](https://github.com/bcdev/xcube-hub/issues).

## Init the GeoDB client

In [2]:
from xcube_geodb.core.geodb import GeoDBClient
geodb = GeoDBClient()
geodb.whoami

'geodb_112a590d-cfaf-4bad-bafc-d597f531e98b'

### Create a Database and Copy some Public Data Across

In [2]:
my_database_name = 'my-urban-h'

In [3]:
geodb.create_database(my_database_name)

<h1>Error: {"hint":null,"details":null,"code":"P0001","message":"Database my-urban-h exists already."}</h1>

In [4]:
geodb.drop_collection(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

<h1>Collection ['my-urban-h_SI001L2_LJUBLJANA_UA2018'] deleted</h1>

In [5]:
geodb.copy_collection(collection='SI001L2_LJUBLJANA_UA2018', new_collection='SI001L2_LJUBLJANA_UA2018', database='eea-urban-atlas', new_database=my_database_name)

<h1>Collection copied from eea-urban-atlas/SI001L2_LJUBLJANA_UA2018 to my-urban-h/SI001L2_LJUBLJANA_UA2018</h1>

In [3]:
geodb.get_all_published_gs()

,collection_id,database,default_style,geojson_url,href,name,preview_url,wfs_url
0,my-urban-h_SI001L2_LJUBLJANA_UA2018,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,my-urban-h_SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
1,my-urban-h_SI001L2_LJUBLJANA_UA2018,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,my-urban-h_SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
2,OSM-Overlay-WMS,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,OSM-Overlay-WMS,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
3,OSM-WMS,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,OSM-WMS,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
4,SRTM30-Colored,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SRTM30-Colored,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
5,SRTM30-Colored-Hillshade,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SRTM30-Colored-Hillshade,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
6,SRTM30-Contour,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SRTM30-Contour,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
7,SRTM30-Hillshade,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SRTM30-Hillshade,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
8,TOPO-OSM-WMS,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,TOPO-OSM-WMS,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...
9,TOPO-WMS,None,None,https://stage.xcube-geodb.brockmann-consult.de...,None,TOPO-WMS,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...


In [6]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

,collection_id,database,default_style,geojson_url,href,name,preview_url,wfs_url
0,SI001L2_LJUBLJANA_UA2018,my-urban-h,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...


## Publish your Collection to the BC WMS Service

In [7]:
geodb.publish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

{'collection_id': 'SI001L2_LJUBLJANA_UA2018',
 'database': 'my-urban-h',
 'default_style': None,
 'geojson_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-h/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban-h:my-urban-h_SI001L2_LJUBLJANA_UA2018&maxFeatures=10&outputFormat=application/json',
 'href': None,
 'name': 'SI001L2_LJUBLJANA_UA2018',
 'preview_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-h/wms?service=WMS&version=1.1.0&request=GetMap&layers=my-urban-h:my-urban-h_SI001L2_LJUBLJANA_UA2018&bbox=4638735,2524481.25,4704972.5,2594978.75&width=690&height=768&srs=EPSG:3035&styles=&format=application/openlayers',
 'wfs_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-h/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban-h%3Amy-urban-h_SI001L2_LJUBLJANA_UA2018&maxFeatures=10&outputFormat=application%2Fvnd.google-earth.kml%2Bxml'}

You can now click on the preview link which will give you an unstyled view of your WMS output.

In [8]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

,collection_id,database,default_style,geojson_url,href,name,preview_url,wfs_url
0,SI001L2_LJUBLJANA_UA2018,my-urban-h,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...


### View the Data

Once you have published you collection, you can use the WMS service to visualise your collection. For this using ipyleaflet. In this example we have used a pre-defined style. You can always provide a custom style using the parameter `sld`. We have provided a class that will pass that parameter to the WMS service. Just replace WMSLayer with SldWMSLayer. The `sld`parameter is given as a URL to an SLD file.

In [30]:
url = f"http://stage.xcube-geodb.brockmann-consult.de/geoserver/{my_database_name}/wms?"


In [31]:
from ipyleaflet import Map, WMSLayer, basemaps

In [32]:
from traitlets import Unicode

class SldWMSLayer(WMSLayer):

    sld = Unicode('').tag(sync=True, o=True)


In [33]:
wmsn = WMSLayer(
    url=url,
    layers=f'{my_database_name}:{my_database_name}_SI001L2_LJUBLJANA_UA2018',
    format='image/png',
    opacity=0.5,
    attribution='Copernicus',
    styles='ljubilana'
#     sld='https://raw.githubusercontent.com/dzelge/test-gha/main/ljubilana.sld'
)

In [34]:
m = Map(basemap=basemaps.CartoDB.Positron, center=(46.0, 14.5), zoom=13)
m.add_layer(wmsn)

m

Map(center=[46.0, 14.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [39]:
geodb.unpublish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

True